In [ ]:
import pandas as pd
import numpy as np
import unicodedata

#Coverte de número por extenso para digito

numeros_extenso = {
    'zero' : 0, 'um' : 1, 'dois' : 2, 'tres' : 3, 'quatro' : 4, 'cinco' : 5, 'seis' : 6, 'sete': 7, 'oito' : 8, 'nove' : 9, 'dez' : 10, "onze": 11,
    "doze": 12, "treze": 13, "quatorze": 14, "catorze": 14,
    "quinze": 15, "dezesseis": 16, "dezessete": 17, "dezoito": 18,
    "dezenove": 19, "vinte": 20, "trinta": 30, "quarenta": 40,
    "cinquenta": 50, "sessenta": 60, "setenta": 70,
    "oitenta": 80, "noventa": 90, 'cem' : 100
}

def converte_extenso_numero(s):
  #Verifica se está como None ou vazio e returna no formato NaN
  if s is None or pd.isna(s):
    return np.nan

  #Elimina espaços antes e depois e converte para letra minúscula
  s = s.strip().lower()

  #Converte cada letra para a versão sem acento e recoloca no lugar
  s = ''.join(letra for letra in unicodedata.normalize('NFD', s)
           if unicodedata.category(letra)!= 'Mn')

  #Pega apenas a primeira palavra
  palavra = s.split()[0]

  #Se já é número ou tem dígitos, extrai
  num = ''.join(letra for letra in palavra if letra.isdigit())
  if num.isdigit():
    return float(num)

  #Verifica no dicionário de extenso
  return float(numeros_extenso.get(palavra, np.nan))


#Remoção de acentos
def replace_accents(s):
  if s is None:
    return s
  return ''.join(
      letra for letra in unicodedata.normalize('NFD', s)
      if unicodedata.category(letra)!= 'Mn'
  )


#Normalização de nomes de colunas
def normaliza_nomes_colunas(df):
  '''Retira espaços antes e depois dos nomes das colunas, converte em minúsculo, tira os acentos, troca espaços ou qualquer símbolo
  que separa as palavras por '_' , e garante que não haja dois ou mais '_' seguidos, além de retirar '_' da frente e de trás das palavras.
  '''
  df.columns = df.columns.astype(str)
  df.columns = (
      df.columns
        .str.strip()
        .str.lower()
        .map(replace_accents)
        .str.replace(r'[^a-z0-9]','_', regex=True)
        .str.replace(r'_+','_', regex=True)
        .str.strip('_')
  )
  return df

#Normalização de texto em colunas

def normaliza_texto(col):
  #Garante que a coluna que vai ser transformada está no formato de string
  col = col.astype(str)


  #Troca espaços vazios, 'nan' com espaços antes ou depois (ou sem), e None com espaço antes ou depois por np.nan
  col = col.replace(
      to_replace= [r'^\s*$', r'^\s*nan\s*$', r'^\s*None\s*$',],
      value = np.nan,
      regex = True
  )

  #Pega as linhas da coluna que não estão vazias
  mask = col.notna()

  '''Tira em todas as linhas da coluna que tem texto (str) os espaços da frente, transforma em minúscula, aplica a função de tirar as letras com acento
  para a versão sem, subistitui tudo que não é letra por '_', garante que não tenha mais de um '_' seguido, e tira '_' do começo ou fim do texto.
  '''
  col.loc[mask] = (
      col[mask]
        .str.strip()
        .str.lower()
        .map(replace_accents)
        .str.replace(r'[^a-z0-9]', '_', regex=True)
        .str.replace(r'_+','_', regex=True)
        .str.strip('_')
  )

  #Coloca como nan em linhas que consistem de espaços
  col = col.replace(r'^\s*$', np.nan, regex= True)
  return col

#Normalização de valores numéricos

def normaliza_valores_numericos(col):
    col = (
        col.astype(str)
            .str.strip()
            .str.replace(r'[^0-9.,-]', '', regex=True)  # mantém apenas números, vírgula, ponto e -
    )

    # Se existe vírgula, assume que ela é decimal → troca por ponto
    col = col.str.replace(',', '.', regex=False)

    # Remove múltiplos pontos seguidos (erro comum)
    col = col.str.replace(r'\.+', '.', regex=True)

    # Remove '-' que não estejam no início
    col = col.str.replace(r'(?<!^)-', '', regex=True)

    # Valores vazios viram NaN
    col = col.replace(r'^\s*$', np.nan, regex=True)

    return col.astype(float)

#WRAPPERS

#Aplica para uma lista de colunas a função normaliza_texto
def normaliza_df_texto(df, colunas):
  for c in colunas:
    df[c] = normaliza_texto(df[c])
  return df

#Aplica para uma lista de colunas a função normaliza_valores_numericos
def normaliza_df_numerico(df,colunas):
    for c in colunas:
      df[c] = normaliza_valores_numericos(df[c])
    return df


#Remover colunas "unnamed"
def remove_colunas_unnamed(df):
  cols = [c for c in df.columns if c.lower().startswith("unnamed")]
  df.drop(columns=cols, inplace=True)
  return df


#Normaliza datas

def normaliza_data(col):
  return pd.to_datetime(col, errors= "coerce", dayfirst= True)


#Pipeline básico

def limpeza_basica(df):
  normaliza_nomes_colunas(df)
  remove_colunas_unnamed(df)
  return df



In [ ]:
from openpyxl import Workbook
import random

#cria a planilha
wb = Workbook()
ws = wb.active
ws.title = "teste"

#Cria cabeçalhos cheios de erros
headers = [" Nome ", "IDADE??", "preço,unitário", "Data  ", None, "###CODIGO###"]
ws.append(headers)

#Linhas Bagunçadas aleatórias
for i in range(30):
  row = [
      random.choice(["Ana", " Beto", "Carlos", None, "123", "??Nome??"]),
      random.choice([25,"vinte ", None, 42, "43a", "","0"]),
      random.choice(["10,50", "R$ 5.00", None, "abc", 7.8]),
      random.choice(["2023-01-0" + str(random.randint(1,9)),
      "32/13/2022", None, "", "lixo", 999]),
      random.choice(["extra", None, "", "lixo", 999]),
      random.choice(["A1","B2", None, "###", 123])
  ]
  ws.append(row)

#Linha vazia no fim
for _ in range(5):
  ws.append([None]*6)

# Salva o arquivo
wb.save("teste_dataframe_sujo.xlsx")
print("Arquivo gerado: teste_dataframe_sujo.xlsx")


Arquivo gerado: teste_dataframe_sujo.xlsx
